In [25]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, find_state_extract, directory_scratch, directory_project


In [106]:
plate_number = 436
plate = get_postion_number(plate_number)
print(plate)
# directory = directory_scratch
directory = directory_project
listdir = os.listdir(directory)
list_dir_interest = [name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
dates_datetime = get_dates_datetime(directory,plate)
len(list_dir_interest)

12


152

In [107]:
plate_number = get_plate_number(plate,dates_datetime[0])
print(0,plate_number)
for i in range(len(list_dir_interest)):
    new_plate_number = get_plate_number(plate,dates_datetime[i])
    if plate_number!=new_plate_number:
        plate_number=new_plate_number
        print(i,plate_number)


0 436


In [128]:
begin = 5
end = begin + 16

print('begin =', dates_datetime[begin], '\n  end =', dates_datetime[end])

begin = 2020-10-01 10:43:00 
  end = 2020-10-05 10:11:00


In [127]:
dates_datetime[5:5+16]

[datetime.datetime(2020, 10, 1, 10, 43),
 datetime.datetime(2020, 10, 1, 14, 43),
 datetime.datetime(2020, 10, 1, 18, 43),
 datetime.datetime(2020, 10, 1, 22, 43),
 datetime.datetime(2020, 10, 2, 2, 42),
 datetime.datetime(2020, 10, 2, 6, 43),
 datetime.datetime(2020, 10, 2, 10, 43),
 datetime.datetime(2020, 10, 2, 14, 43),
 datetime.datetime(2020, 10, 2, 18, 43),
 datetime.datetime(2020, 10, 2, 22, 43),
 datetime.datetime(2020, 10, 3, 2, 43),
 datetime.datetime(2020, 10, 3, 6, 43),
 datetime.datetime(2020, 10, 3, 10, 43),
 datetime.datetime(2020, 10, 3, 14, 43),
 datetime.datetime(2020, 10, 3, 18, 43),
 datetime.datetime(2020, 10, 3, 22, 42)]

In [17]:
dates_datetime[begin:end]

[datetime.datetime(2020, 8, 14, 1, 36),
 datetime.datetime(2020, 8, 14, 5, 36),
 datetime.datetime(2020, 8, 14, 9, 36),
 datetime.datetime(2020, 8, 14, 13, 36),
 datetime.datetime(2020, 8, 14, 17, 36),
 datetime.datetime(2020, 8, 14, 21, 36),
 datetime.datetime(2020, 8, 15, 1, 36),
 datetime.datetime(2020, 8, 15, 5, 36),
 datetime.datetime(2020, 8, 15, 9, 36),
 datetime.datetime(2020, 8, 15, 13, 36),
 datetime.datetime(2020, 8, 15, 17, 37),
 datetime.datetime(2020, 8, 15, 21, 36),
 datetime.datetime(2020, 8, 16, 1, 36),
 datetime.datetime(2020, 8, 16, 5, 36),
 datetime.datetime(2020, 8, 16, 9, 36),
 datetime.datetime(2020, 8, 16, 13, 36),
 datetime.datetime(2020, 8, 16, 17, 37),
 datetime.datetime(2020, 8, 16, 21, 37)]

In [110]:
find_state(plate, begin, end, directory,True)

('/Img/TileConfiguration.txt.registered',
 [(datetime.datetime(2020, 10, 12, 22, 15), 64),
  (datetime.datetime(2020, 10, 13, 2, 14), 65),
  (datetime.datetime(2020, 10, 13, 6, 14), 66),
  (datetime.datetime(2020, 10, 13, 10, 15), 67),
  (datetime.datetime(2020, 10, 14, 2, 15), 68),
  (datetime.datetime(2020, 10, 14, 6, 15), 69),
  (datetime.datetime(2020, 10, 14, 10, 15), 70),
  (datetime.datetime(2020, 10, 14, 18, 15), 71),
  (datetime.datetime(2020, 10, 14, 22, 15), 72),
  (datetime.datetime(2020, 10, 15, 2, 15), 73),
  (datetime.datetime(2020, 10, 15, 6, 15), 74),
  (datetime.datetime(2020, 10, 15, 10, 15), 75),
  (datetime.datetime(2020, 10, 15, 14, 15), 76),
  (datetime.datetime(2020, 10, 15, 18, 15), 77),
  (datetime.datetime(2020, 10, 15, 22, 15), 78),
  (datetime.datetime(2020, 10, 16, 2, 15), 79),
  (datetime.datetime(2020, 10, 16, 6, 15), 80),
  (datetime.datetime(2020, 10, 16, 10, 15), 81),
  (datetime.datetime(2020, 10, 16, 14, 15), 82),
  (datetime.datetime(2020, 10, 16, 

In [111]:
find_state_extract(plate, begin, end, directory)

('/Analysis/nx_graph_pruned.p',
 [(datetime.datetime(2020, 9, 29, 14, 36), 0),
  (datetime.datetime(2020, 9, 29, 18, 35), 1),
  (datetime.datetime(2020, 9, 29, 22, 36), 2),
  (datetime.datetime(2020, 9, 30, 18, 36), 3),
  (datetime.datetime(2020, 9, 30, 22, 36), 4),
  (datetime.datetime(2020, 10, 1, 10, 43), 5),
  (datetime.datetime(2020, 10, 1, 14, 43), 6),
  (datetime.datetime(2020, 10, 1, 18, 43), 7),
  (datetime.datetime(2020, 10, 1, 22, 43), 8),
  (datetime.datetime(2020, 10, 2, 2, 42), 9),
  (datetime.datetime(2020, 10, 2, 6, 43), 10),
  (datetime.datetime(2020, 10, 2, 10, 43), 11),
  (datetime.datetime(2020, 10, 2, 14, 43), 12),
  (datetime.datetime(2020, 10, 2, 18, 43), 13),
  (datetime.datetime(2020, 10, 2, 22, 43), 14),
  (datetime.datetime(2020, 10, 3, 2, 43), 15),
  (datetime.datetime(2020, 10, 3, 6, 43), 16),
  (datetime.datetime(2020, 10, 3, 10, 43), 17),
  (datetime.datetime(2020, 10, 3, 14, 43), 18),
  (datetime.datetime(2020, 10, 3, 18, 43), 19),
  (datetime.datetime(2

In [6]:
get_dirname(dates_datetime[14], plate)

'20200921_0805_Plate10'

In [131]:
start = begin + 12
finish = start + 1
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[start:finish+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [132]:
plt.close('all')
start = 0
finish = start + 1
for i in range(start,finish):
    plot_t_tp1([], [], None, None, skels[i], ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
start = begin + 10
finish = start +2
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[start:finish+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5, 5), np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap = directory + directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_realigned_compressed.mat')
    skel = skel_info['skeleton']
#     skels.append(skel)
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)
plt.close('all')
start = 0
finish = start + 1
for i in range(start, finish):
    plot_t_tp1([],[],None,None,skels[i],skels[i+1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Create graphs***

In [133]:
num_parallel = 20
time = '30:00'
args=[plate, directory]
run_parallel('extract_nx_graph.py',args,begin,end,num_parallel,time,'extract_nx')

***Extract Width***

In [66]:
num_parallel = 16
time = '1:00:00'
args = [plate,directory]
run_parallel('extract_width.py',args,begin,end,num_parallel,time,'extract_width')

***Identify Nodes***

In [105]:
num_parallel = 1
time = '12:00:00'
args=[plate,begin,end, directory]
run_parallel('extract_nodes_surf.py',args,0,0,num_parallel,time,'node_id')

***Hyphae extraction***

In [78]:
num_parallel = 1
time = '2:00:00'
args = [plate, begin, end, directory]
run_parallel('hyphae_extraction.py',args,0,0,num_parallel,time,'hyphae')